In [89]:
# tensorflow 2.0.0
import tensorflow as tf 
import keras
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
import numpy as np
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.models import Model


In [90]:
# to scale images consider using 
# im = im.resize(size,Image.LANCZOS) #if your images are not already the size you want    


from PIL import Image
import random
def load_data():
    from PIL import Image
    import random
    input_size_p = 40754
    input_size_n = 1723
    p_data_list = [] # positive data
    n_data_list = [] # negative data
    for i in range(input_size_p):
        im = Image.open('p_images/nifH'+str(i)+'.jpg')
    ##        Using img_to_array instead of np.array, requires to add (0,0) third tuple for the pad dimensions
        arr = img_to_array(im)
        if arr.shape[0] < 847:
            dif = 847 - arr.shape[0]
            arr = np.pad(arr,((0, dif), (0, 0), (0,0)),'constant', constant_values=(0))
        p_data_list.append(arr)
    #     p_data = np.array(p_data_list)
    p_labels = [1] * 40754

    random.shuffle(p_data_list)
    X_train_list, X_test_list = p_data_list[:39754], p_data_list[39745:]
    Y_train_list, Y_test_list = p_labels[:39754], p_labels[39745:]

    # negative data loop
    for i in range(input_size_n):
        im = Image.open('n_images/nifH'+str(i)+'.jpg')
        arr = img_to_array(im)
        if arr.shape[0] < 847:
            dif = 847 - arr.shape[0]
            arr = np.pad(arr,((0, dif), (0, 0),(0,0)),'constant', constant_values=(0))
        n_data_list.append(arr)
    #     n_data = np.array(n_data_list)
    n_labels = [0] * 1723

    random.shuffle(n_data_list)
    n_X_train_list, n_X_test_list = n_data_list[:1551], n_data_list[1551:]
    n_Y_train_list, n_Y_test_list = n_labels[:1551], n_labels[1551:]

    X_train_list = X_train_list + n_X_train_list
    X_test_list = X_test_list + n_X_test_list
    Y_train_list = Y_train_list+ n_Y_train_list
    Y_test_list = Y_test_list + n_Y_test_list

    X_train = np.dstack(X_train_list)
    X_test = np.dstack(X_test_list)
    Y_train = np.dstack(Y_train_list)
    Y_test = np.dstack(Y_test_list)
    
    return X_train, Y_train, X_test, Y_test

In [91]:
len(X_train_list[39754])

847

In [92]:
# im.show()
X_train, Y_train, X_test, Y_test = load_data()


In [93]:
##TODO
## the data is a 'list' of 'np arrays', it should probably be a np instead of a list
## model
## scale images?
## when saving the model, also save the train and test data sets with labels becasue they are shuffeled randomly
# X_train_tensor = X_train
X_train.shape
X_train = np.rollaxis(X_train,-1)
X_train.shape
X_train = X_train.reshape((41305, 847, 7,1))
X_train.shape



# r = X_train.reshape(101,847,7)
# r.shape

(41305, 847, 7, 1)

In [94]:
#X images shape [x, 7, 1]
## dimensions of the tensors X_train [39755, x, 7, 1] Y_train [39755] X_test  [2010, x, 7, 1] Y_test  [2010]
# len(X_train)
# len(X_test)

In [95]:
# maximim sequence length
m = 0
for i in range(len(X_train)-1):
    m = max(m, len(X_train[i]))
print(m, i)
# max= 847

847 41303


In [96]:
# miniimum sequence length
m = 1000
for i in range(len(X_train)-1):
    m = min(m, len(X_train[i]))
print(m, i)
# min= 37

847 41303


In [200]:
def nifhmodel(input_shape):
    # defining input
    X_input = Input(input_shape)
    print(X_input.shape)
    # CONV 1
    X = Conv2D(4, (6, 4))(X_input) # try other values for i in f = (i*3, 4)
    X = Activation('relu')(X)
    print(X.shape)
    # CONV 2
    X = Conv2D(6, (6, 1))(X_input) # try other values for i in f = (i*3, 4)
    X = Activation('relu')(X)
    print(X.shape)
    # Max
    X = MaxPooling2D((2, 2))(X)
    print(X.shape)
    # CONV 3
    X = Conv2D(6, (9, 1))(X_input) # try other values for i in f = (i*3, 4)
    X = Activation('relu')(X)
    print(X.shape)
    # Max
    X = MaxPooling2D((2, 2), name='max_pool_2')(X)
    print(X.shape)
    # CONV 4
    X = Conv2D(12, (9, 4))(X_input) # try other values for i in f = (i*3, 4)
    X = BatchNormalization(axis = 3, name = 'bn0')(X)
    X = Activation('relu')(X)
    print(X.shape)
    # Max
    X = MaxPooling2D((2, 1), name='max_pool_3')(X)
    
    # CONV 5
    X = Conv2D(12, (9, 4))(X_input) # try other values for i in f = (i*3, 4)
    X = Activation('relu')(X)
    print("Afetr conv2 " + str(X.shape))


    # Max
    X = MaxPooling2D((2, 1), name='max_pool_4')(X)
    print("After max pooling "+ str(X.shape))


    X = Flatten()(X)
    print("After flatten, before dense" + str(X.shape))
    X = Dense(1, activation='sigmoid', name='dense_1')(X)
    print("After dense" + str(X.shape))
    model = Model(inputs = X_input, outputs = X)
    
    return model

In [201]:
# num_data = X_train.shape[0]
# rows = X_train[0].shape[0]
# columns = X_train[0].shape[1]



In [202]:
nifhmodel = nifhmodel(X_train[0].shape)

nifhmodel.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ["accuracy"])

nifhmodel.fit(x = X_train, y = Y_train, epochs = 5, batch_size = 16)

preds = nifhmodel.evaluate(x = X_test, y = Y_test)

print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

(None, 847, 7, 1)
(None, 842, 4, 4)
(None, 842, 7, 6)
(None, 421, 3, 6)
(None, 839, 7, 6)
(None, 419, 3, 6)
(None, 839, 4, 12)
Afetr conv2 (None, 839, 4, 12)
After max pooling (None, 419, 4, 12)
After flatten, before dense(None, None)
After dense(None, 1)


ValueError: Error when checking target: expected dense_1 to have 2 dimensions, but got array with shape (1, 1, 41305)

In [ ]:
ValueError: Error when checking target: expected dense_1 to have 2 dimensions, but got array with shape (1, 1, 41305)
